<a href="https://colab.research.google.com/github/MiguelQuiceno/FHE-ML/blob/main/Aprendiendo_Zama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pip wheel setuptools
!pip install concrete-python

  Using cached concrete_python-2.9.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (12 kB)
  Using cached z3_solver-4.13.0.0-py2.py3-none-manylinux2014_x86_64.whl.metadata (757 bytes)
Using cached concrete_python-2.9.0-cp311-cp311-manylinux_2_28_x86_64.whl (77.0 MB)
Using cached z3_solver-4.13.0.0-py2.py3-none-manylinux2014_x86_64.whl (57.3 MB)


In [2]:
"""
The basic workflow of computation is as follows:

Define the function you want to compute

Compile the function into a Concrete Circuit

Use the Circuit to perform homomorphic evaluation"""

from concrete import fhe

def add(x, y):
  return x + y

compiler = fhe.Compiler(add, {'x': 'encrypted', 'y':'encrypted'})

inputset = [(2, 3), (0, 0)]

print(f'Compilation...')
circuit = compiler.compile(inputset)

print(f'Key generation...')
circuit.keygen()

print(f'Homomorphic evaluation...')

encrypted_x, encrypted_y = circuit.encrypt(2, 6)
encrypted_result = circuit.run(encrypted_x, encrypted_y)
result = circuit.decrypt(encrypted_result)

print(result)




Compilation...
Key generation...
Homomorphic evaluation...
8


In [12]:
# Otra forma de compilar con un decorador

from math import log

@fhe.compiler({'x': 'encrypted', 'y': 'encrypted', 'z': 'encrypted'})
def f(x, y, z):
  return x * y * z

inputset = [(2, 1, 3), (3, 3, 4)]
circuit = f.compile(inputset)

circuit.encrypt_run_decrypt(3, 1, 3)



9

In [15]:
#Parece que no se pueden encriptar funciones

@fhe.compiler({'f': 'encrypted', 'a': 'encrypted', 'b': 'encrypted'}) # Remove 'f' from encryption
def g(f, a, b):
  return f(a) + b

# Define functions outside inputset
def square(x):
    return x ** 2

def log_func(x):
    return log(x)

inputset = [(square, 1, 3), (log_func, 3, 4)] # Use function names instead of lambda functions
circuit = g.compile(inputset)

circuit.encrypt_run_decrypt(square, 2, 6) # Pass the function name directly


ValueError: Concrete cannot represent <function square at 0x79e8ef0bf560>

In [19]:
#El framework también permite hacer composición

@fhe.compiler({'counter': 'encrypted'})
def increment(counter):
  return (counter + 1) % 100

print('Compiling increment function')
increment_fhe = increment.compile(list(range(0, 100)), composable = True)

print('Generating keyset')
increment_fhe.keygen()

print('Encrypting the initial counter value')
counter = 0
counter_enc = increment_fhe.encrypt(counter)

print(f"| iteration || decrypted | cleartext |")

for i in range(10):
  counter_enc = increment_fhe.run(counter_enc)
  counter = increment(counter)
  counter_dec = increment_fhe.decrypt(counter_enc)
  print(f"|     {i}     || {counter_dec:<9} | {counter:<9} |")

Compiling increment function
Generating keyset
Encrypting the initial counter value
| iteration || decrypted | cleartext |
|     0     || 1         | 1         |
|     1     || 2         | 2         |
|     2     || 3         | 3         |
|     3     || 4         | 4         |
|     4     || 5         | 5         |
|     5     || 6         | 6         |
|     6     || 7         | 7         |
|     7     || 8         | 8         |
|     8     || 9         | 9         |
|     9     || 10        | 10        |


In [17]:
counter_enc

In [23]:
# También podemos componer con circuitos de múltiples inputs y múltiples outputs

def noise_reset(x):
  return fhe.univariate(lambda x: x)(x)

@fhe.compiler({'n1th': 'encrypted', 'nth':'encrypted'})
def fib(n1th, nth):
  return noise_reset(nth), noise_reset(n1th + nth)

print('Compiling fib function')
inputset = list(zip(range(0, 100), range(0, 100)))
fib_fhe = fib.compile(inputset, composable = True)

print('Generating keyset')
fib_fhe.keygen()

print('Encrypting inital values')
n1th = 1
nth = 2
(n1th_enc, nth_enc) = fib_fhe.encrypt(n1th, nth)

for i in range(10):

  (n1th_enc, nth_enc) = fib_fhe.run(n1th_enc, nth_enc)
  (n1th, nth) = fib(n1th, nth)

  (n1th_dec, nth_dec) = fib_fhe.decrypt(n1th_enc, nth_enc)

Compiling fib function
Generating keyset
Encrypting inital values


In [25]:
# Para hacer composición, debemos ir refrescando el ruido

def noise_reset(x):
  return fhe.univariate(lambda x: x)(x)

@fhe.compiler({'counter': 'encrypted'})
def double(counter):
  return noise_reset(counter * 2)

print('Compilando double')
inputset = range(0, 100)
double_fhe = double.compile(inputset, composable = True)

Compilando double


In [26]:
print('generando keyset')

double_fhe.keygen()

generando keyset


In [30]:
x_enc = double_fhe.encrypt(20)
result = double_fhe.run(x_enc)
double_fhe.decrypt(result)

40